# 循环神经网络(RNN)详解

## 一、什么是RNN?

循环神经网络(Recurrent Neural Network, RNN)是一种专门用于处理**序列数据**的神经网络。

### RNN的核心特点

- **记忆能力**:RNN具有记忆功能,能够保存之前的信息并影响当前的输出
- **参数共享**:所有时间步共享相同的权重参数
- **可变长度输入**:能够处理任意长度的序列数据
- **时序依赖**:考虑数据之间的时间顺序关系

### 为什么需要RNN?

传统的前馈神经网络(如全连接网络、CNN)有一个局限:它们假设输入之间是独立的,无法处理具有时序关系的数据。而在很多实际场景中,数据是有时间顺序的:

- **自然语言处理**:句子中词的顺序很重要,"我喜欢你"和"你喜欢我"意思完全不同
- **时间序列预测**:股票价格、天气预报等,当前的值依赖于历史数据
- **语音识别**:语音信号是连续的时序数据
- **视频处理**:视频是连续的图像帧序列

## 二、RNN的工作原理

### 隐状态(Hidden State)

RNN的核心概念是**隐状态**(Hidden State),它像是神经网络的"记忆":

- 在每个时间步,RNN接收**当前输入**和**上一时刻的隐状态**
- 计算得到**当前输出**和**当前隐状态**
- 当前隐状态会传递到下一个时间步

数学表达式:

$$
h_t = \tanh(W_{hh} \cdot h_{t-1} + W_{xh} \cdot x_t + b_h)
$$

$$
y_t = W_{hy} \cdot h_t + b_y
$$

其中:
- $h_t$: 当前时刻的隐状态
- $h_{t-1}$: 上一时刻的隐状态(记忆)
- $x_t$: 当前时刻的输入
- $y_t$: 当前时刻的输出
- $W_{hh}, W_{xh}, W_{hy}$: 权重矩阵(在所有时间步共享)
- $b_h, b_y$: 偏置项

### 多层RNN

为了增强模型的表达能力,可以堆叠多个RNN层:
- 第一层的输出作为第二层的输入
- 每一层都有自己的隐状态
- 可以学习不同层次的特征表示

## 三、本示例的学习目标

1. 理解PyTorch中`nn.RNN`的参数含义
2. 掌握RNN的输入输出张量形状
3. 理解隐状态的作用和维度
4. 学习如何使用RNN处理序列数据

---

In [19]:
# ============================================
# 导入必要的库
# ============================================

import torch              # PyTorch核心库,用于张量计算
import torch.nn as nn     # PyTorch神经网络模块,提供RNN等层
import torch.optim as optim  # PyTorch优化器模块,提供SGD、Adam等优化算法

## 步骤1:创建RNN层

### nn.RNN详解

`nn.RNN`是PyTorch提供的标准RNN层实现。

### 参数说明

```python
nn.RNN(input_size, hidden_size, num_layers, ...)
```

**核心参数:**

1. **input_size=8**:输入特征的维度
   - 表示每个时间步的输入向量有8个特征
   - 例如:在词嵌入中,如果词向量维度是8,则input_size=8
   - 在时间序列中,如果每个时刻有8个传感器读数,则input_size=8

2. **hidden_size=16**:隐状态的维度
   - 表示RNN内部的"记忆"用16维向量表示
   - 也是输出向量的维度
   - hidden_size越大,模型容量越大,但计算成本也越高
   - 常见取值:64、128、256、512等

3. **num_layers=2**:RNN的层数
   - 表示堆叠2层RNN
   - 第1层的输出作为第2层的输入
   - 多层RNN能够学习更复杂的模式和更高层次的抽象
   - 层数越多,表达能力越强,但也更容易过拟合

**其他常用参数(本例未使用):**

- `nonlinearity='tanh'`:激活函数,默认是tanh,也可以选择'relu'
- `bias=True`:是否使用偏置项
- `batch_first=False`:输入输出的维度顺序
  - False(默认):(seq_len, batch, input_size)
  - True:(batch, seq_len, input_size)
- `dropout=0`:层间的dropout概率(当num_layers>1时有效)
- `bidirectional=False`:是否使用双向RNN

### RNN层的权重参数

创建后,RNN层内部包含以下可学习的参数:
- `weight_ih_l[k]`:第k层的输入到隐状态的权重,形状(hidden_size, input_size)
- `weight_hh_l[k]`:第k层的隐状态到隐状态的权重,形状(hidden_size, hidden_size)
- `bias_ih_l[k]`:第k层输入的偏置
- `bias_hh_l[k]`:第k层隐状态的偏置

这些参数在训练过程中会通过反向传播不断更新优化。

In [20]:
# ============================================
# 步骤1:创建RNN层
# ============================================

# 创建一个两层的RNN网络
# input_size=8:  每个时间步的输入特征维度为8
# hidden_size=16: 隐状态的维度为16(也是输出维度)
# num_layers=2:   堆叠2层RNN,增强模型表达能力

rnn = nn.RNN(input_size=8, hidden_size=16, num_layers=2)

# RNN的结构:
# 输入(8维) -> RNN第1层(16维隐状态) -> RNN第2层(16维隐状态) -> 输出(16维)
# 每一层都维护自己的隐状态,层与层之间隐状态独立

## 步骤2:准备输入数据和初始隐状态

### RNN的输入格式

RNN需要两个输入:
1. **输入序列**(input):待处理的序列数据
2. **初始隐状态**(h0):RNN的初始记忆状态

### 输入张量的形状

#### input形状:(seq_len, batch, input_size)

当`batch_first=False`(默认)时,输入形状为`(seq_len, batch, input_size)`:

- **seq_len=3**:序列长度,表示有3个时间步
  - 例如:一个句子有3个词
  - 或者:一段时间序列有3个时刻的数据

- **batch=3**:批次大小,表示同时处理3个样本
  - 批处理可以加速训练
  - 3个样本可以并行处理

- **input_size=8**:每个时间步的特征维度
  - 必须与RNN定义时的input_size一致
  - 例如:每个词用8维向量表示

**示例理解**:`input[0, 1, :]`表示:
- 第1个时间步(第1个词)
- 第2个样本(第2个句子)
- 该词的8维特征向量

#### h0形状:(num_layers, batch, hidden_size)

初始隐状态的形状为`(num_layers, batch, hidden_size)`:

- **num_layers=2**:RNN的层数
  - 每一层都需要自己的初始隐状态
  - hx[0]是第1层的初始隐状态
  - hx[1]是第2层的初始隐状态

- **batch=3**:批次大小,必须与input的batch一致
  - 每个样本都有自己的隐状态

- **hidden_size=16**:隐状态的维度
  - 必须与RNN定义时的hidden_size一致

### torch.randn()函数

`torch.randn()`生成服从标准正态分布(均值0,标准差1)的随机张量:
- 在实际应用中,初始隐状态通常初始化为全0:`torch.zeros(2, 3, 16)`
- 这里使用随机初始化只是为了演示
- 输入数据通常来自实际数据集,而不是随机生成

In [21]:
# ============================================
# 步骤2:准备输入数据和初始隐状态
# ============================================

# 1. 创建输入序列
# 形状:(seq_len=3, batch=3, input_size=8)
# - 序列长度为3:表示有3个时间步(例如句子有3个词)
# - 批次大小为3:同时处理3个样本(例如3个句子)
# - 特征维度为8:每个时间步的输入是8维向量
# torch.randn()生成随机数据,实际应用中应该是真实的序列数据
input = torch.randn(3, 3, 8)

# 2. 创建初始隐状态
# 形状:(num_layers=2, batch=3, hidden_size=16)
# - 层数为2:因为RNN有2层,每层都需要初始隐状态
# - 批次大小为3:必须与input的batch一致
# - 隐状态维度为16:必须与RNN定义的hidden_size一致
# 初始隐状态相当于RNN的"初始记忆",通常初始化为0或随机值
hx = torch.randn(2, 3, 16)

# 打印形状以验证
print(input.shape)   # 输出: torch.Size([3, 3, 8])
print(hx.shape)      # 输出: torch.Size([2, 3, 16])

# 形状解释:
# input[t, b, f] 表示: 第t个时间步,第b个样本,第f个特征
# hx[l, b, h] 表示: 第l层,第b个样本,第h个隐状态维度

torch.Size([3, 3, 8])
torch.Size([2, 3, 16])


## 步骤3:RNN前向传播

### 前向传播过程

调用RNN进行前向传播:
```python
output, hn = rnn(input, hx)
```

**输入:**
- `input`:输入序列,形状(3, 3, 8)
- `hx`:初始隐状态,形状(2, 3, 16)

**输出:**
- `output`:所有时间步的输出,形状(3, 3, 16)
- `hn`:最后一个时间步的隐状态,形状(2, 3, 16)

### 内部计算过程详解

RNN会按照时间步顺序处理输入序列:

**时间步1(t=0):**
1. 取出input[0]作为第1个时间步的输入,形状(3, 8)
2. 第1层RNN:
   - 输入:input[0]和hx[0](第1层初始隐状态)
   - 计算:h1_0 = tanh(W * input[0] + U * hx[0] + b)
   - 输出:第1层的隐状态h1_0,形状(3, 16)
3. 第2层RNN:
   - 输入:h1_0(第1层的输出)和hx[1](第2层初始隐状态)
   - 计算:h2_0 = tanh(W * h1_0 + U * hx[1] + b)
   - 输出:第2层的隐状态h2_0,形状(3, 16)
4. output[0] = h2_0(最后一层的输出)

**时间步2(t=1):**
1. 取出input[1]作为第2个时间步的输入
2. 第1层:使用input[1]和上一步的h1_0,计算得到h1_1
3. 第2层:使用h1_1和上一步的h2_0,计算得到h2_1
4. output[1] = h2_1

**时间步3(t=2):**
1. 取出input[2]作为第3个时间步的输入
2. 第1层:使用input[2]和h1_1,计算得到h1_2
3. 第2层:使用h1_2和h2_1,计算得到h2_2
4. output[2] = h2_2

### 输出解释

#### output形状:(seq_len, batch, hidden_size)

`output`包含了**每个时间步最后一层RNN的输出**:

- **形状:(3, 3, 16)**
  - 3个时间步:output[0], output[1], output[2]
  - 3个样本:每个时间步都有3个样本的输出
  - 16维输出:等于hidden_size

- **用途:**
  - 序列标注任务:每个时间步都需要输出(如词性标注)
  - 序列到序列:encoder-decoder架构
  - 注意力机制:需要所有时间步的输出

#### hn形状:(num_layers, batch, hidden_size)

`hn`是**最后一个时间步每一层的隐状态**:

- **形状:(2, 3, 16)**
  - 2层:hn[0]是第1层最后的隐状态,hn[1]是第2层最后的隐状态
  - 3个样本
  - 16维隐状态

- **特殊关系:**
  - `hn[-1]`(最后一层的最后隐状态)等于`output[-1]`(最后时间步的输出)
  - 即:hn[1] == output[2]

- **用途:**
  - 文本分类:只需要最后的隐状态表示整个序列
  - 多层RNN堆叠:将hn传递给下一个RNN
  - 序列生成:作为decoder的初始状态

### 实际应用场景

**1. 文本分类(只用hn):**
```python
output, hn = rnn(input, h0)
final_hidden = hn[-1]  # 取最后一层的隐状态
logits = classifier(final_hidden)  # 分类
```

**2. 序列标注(用output):**
```python
output, hn = rnn(input, h0)
# output的每个时间步都进行预测
predictions = classifier(output)  # 形状:(seq_len, batch, num_classes)
```

**3. Seq2Seq(用hn作为decoder初始状态):**
```python
# Encoder
encoder_output, encoder_hn = encoder_rnn(source, h0)
# Decoder使用encoder的最后隐状态作为初始状态
decoder_output, decoder_hn = decoder_rnn(target, encoder_hn)
```

In [22]:
# ============================================
# 步骤3:RNN前向传播
# ============================================

# 将输入序列和初始隐状态传入RNN
# RNN会按时间步顺序处理输入,同时更新隐状态

# 调用RNN进行前向传播
# 输入: input(3,3,8) - 输入序列
#      hx(2,3,16) - 初始隐状态
# 输出: output - 每个时间步的输出
#      hn - 最后一个时间步的隐状态
output, hn = rnn(input, hx)

# 打印输出形状
print(output.shape)  # 输出: torch.Size([3, 3, 16])
print(hn.shape)      # 输出: torch.Size([2, 3, 16])

# ============================================
# 形状解释
# ============================================

# output形状:(seq_len=3, batch=3, hidden_size=16)
# - 包含所有3个时间步的输出
# - output[0]:第1个时间步,所有样本的输出,形状(3,16)
# - output[1]:第2个时间步,所有样本的输出,形状(3,16)
# - output[2]:第3个时间步,所有样本的输出,形状(3,16)
# - output实际上是最后一层RNN在每个时间步的隐状态

# hn形状:(num_layers=2, batch=3, hidden_size=16)
# - 包含最后一个时间步每一层的隐状态
# - hn[0]:第1层RNN在最后时间步的隐状态,形状(3,16)
# - hn[1]:第2层RNN在最后时间步的隐状态,形状(3,16)
# - 注意:hn[1](第2层最后的隐状态)等于output[2](最后时间步的输出)

# ============================================
# 内部计算流程(简化理解)
# ============================================

# 时间步0: input[0] + hx[0] -> h1_0 (第1层)
#                  h1_0 + hx[1] -> h2_0 (第2层) -> output[0]
#
# 时间步1: input[1] + h1_0 -> h1_1 (第1层)
#                  h1_1 + h2_0 -> h2_1 (第2层) -> output[1]
#
# 时间步2: input[2] + h1_1 -> h1_2 (第1层) -> hn[0]
#                  h1_2 + h2_1 -> h2_2 (第2层) -> output[2] = hn[1]

# 可以验证: output[-1]和hn[-1]应该相等(最后一层的最后时间步)
# print(torch.allclose(output[-1], hn[-1]))  # 输出: True

torch.Size([3, 3, 16])
torch.Size([2, 3, 16])


---

## 总结:RNN的关键概念

### 核心要点

| 概念 | 说明 | 形状 |
|------|------|------|
| 输入序列(input) | 待处理的时序数据 | (seq_len, batch, input_size) |
| 初始隐状态(h0) | RNN的初始记忆 | (num_layers, batch, hidden_size) |
| 输出序列(output) | 每个时间步的输出 | (seq_len, batch, hidden_size) |
| 最终隐状态(hn) | 最后时间步的隐状态 | (num_layers, batch, hidden_size) |

### RNN的三个关键维度

1. **时间维度(seq_len)**:序列有多长,有多少个时间步
2. **批次维度(batch)**:同时处理多少个样本
3. **特征维度(input_size/hidden_size)**:每个数据点的特征数

### RNN的记忆机制

```
时间步0:  input[0] + h_0 -> h_1 -> output[0]
                      ↓
时间步1:  input[1] + h_1 -> h_2 -> output[1]
                      ↓
时间步2:  input[2] + h_2 -> h_3 -> output[2]
```

- 每个时间步的输出都依赖于当前输入和之前的隐状态
- 隐状态像"记忆"一样,将历史信息传递到未来
- 这使RNN能够捕捉序列中的时序依赖关系

### RNN的优势与局限

**优势:**
- ✓ 能处理任意长度的序列
- ✓ 参数共享,模型大小不随序列长度增长
- ✓ 能捕捉时序依赖关系
- ✓ 计算时考虑历史信息

**局限:**
- ✗ **梯度消失/爆炸**:难以学习长期依赖
- ✗ **串行计算**:无法并行处理,训练慢
- ✗ **记忆衰减**:时间步太长会遗忘早期信息

### RNN的改进变体

为了解决标准RNN的局限,研究者提出了多种改进:

**1. LSTM(Long Short-Term Memory)**
- 引入门控机制:输入门、遗忘门、输出门
- 能够更好地捕捉长期依赖
- PyTorch:`nn.LSTM()`

**2. GRU(Gated Recurrent Unit)**
- LSTM的简化版本,只有2个门
- 参数更少,训练更快
- PyTorch:`nn.GRU()`

**3. 双向RNN(Bidirectional RNN)**
- 同时从前向后和从后向前处理序列
- 能同时利用过去和未来的信息
- PyTorch:`nn.RNN(bidirectional=True)`

### 实际应用建议

1. **初学者**:先掌握基本RNN原理,理解时序建模思想
2. **实际项目**:优先使用LSTM或GRU,性能更好
3. **长序列**:考虑Transformer架构,解决了RNN的并行化问题
4. **短序列**:RNN/LSTM/GRU都可以,选择最简单的
5. **双向需求**:文本分类、序列标注等任务可用双向RNN

### 下一步学习

- 学习LSTM和GRU的门控机制
- 了解Seq2Seq模型和注意力机制
- 实践文本分类、情感分析等应用
- 探索Transformer等现代序列模型